In [49]:
%matplotlib inline
import sys, os
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt

In [50]:
%matplotlib inline
import sys,os
import colors
import numpy as np
import matplotlib.cm
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import h5py
import pandas as pd
from datetime import datetime
import matplotlib.image as mpimg
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

In [51]:
import os


'/home/rk/.local/share/Trash/files/Case5_III_C_3yrs_testsd.xml.demo'

In [52]:
os.chdir('/home/rk/ats_rk/testing/ats-demos/rk_model/Case5_III_finaltest')

In [53]:
os.getcwd()

'/home/rk/ats_rk/testing/ats-demos/rk_model/Case5_III_finaltest'

In [54]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import colors
import matplotlib.cm
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

In [55]:
import os
import subprocess
import shutil
import fileinput
import time

In [56]:
file_name = 'Case5_III_C_3yrs_testsd'

In [57]:
# Making a test directory 
os.system('mkdir testsd.demo')

0

In [58]:
os.chdir('testsd.demo')

In [ ]:
# Running ats command
os.system(f"ats --xml_file=../{file_name}.xml")

In [44]:
os.chdir('..')

In [45]:
file_name

'Case5_III_C_3yrs_testsd'

In [46]:
def ats_run(file_name):
    """
    Function that calls & reproduces the command line options for executing ats. It involves mainly the following steps:
    1. Creating a demo directory : mkdir output_file_name.demo
    2. 'cd' into the demo directory : cd out_file_name.demo
    3. Excecuting the ats command : ats --xml_file=../input_file_name.xml &>out.log
    
    Note: To execute the command - Please be in the directory where the input xml file is present.
    
    Parameters:
    
    -----
    INPUT
    
    file_name : string
    
    The input xml file name and the directory name (both are the same). Ex: infiltration
    
    
    -----
    OUTPUT:
    
    Runs the ats command and dumps all the outputs in file_name_i.demo
    
    """
    # Removing the directory if it exists
    if os.path.isdir(f'{file_name}.demo/'):
        shutil.rmtree(f'{file_name}.demo/')
    
    # Making a new directory
    os.mkdir(f'{file_name}.demo')
    
    # Changing the directory to the demo directory
    os.chdir(f'{file_name}.demo/')
    
    # Running the ats command
    ats_command = f"ats --xml_file=../{file_name}.xml >out.log"
    
    os.system(ats_command)
    #output = os.popen(ats_command).read()
    
    #return output
    
    

In [28]:
output = ats_run(file_name)

In [47]:
os.getcwd()

'/home/rk/.local/share/Trash/files/Case5_III_C_3yrs_testsd.xml.demo'

In [22]:
os.chdir('..')

In [31]:
os.getcwd()

'/home/rk/ats_rk/testing/ats-demos/rk_model/Case5_III_finaltest/Case5_III_C_3yrs_testsd.xml.demo'

In [33]:
ats_command = f"ats --xml_file=../{file_name}.xml"
    
os.system(ats_command)

256